In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -U transformers
! pip install datasets
! pip install nltk
! pip install git+https://github.com/google-research/bleurt.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-7fexeqr2
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-7fexeqr2
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (set

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset,DataCollatorForLanguageModeling, AutoModelWithLMHead, TrainingArguments, Trainer
import tensorflow as tf
import datasets
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
nltk.download('punkt')
bleurt = load_metric('bleurt')
bleu = load_metric('bleu')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-5-686576ad6b67>:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleurt = load_metric('bleurt')


In [ ]:
# File paths
file_paths = [
    '/content/drive/MyDrive/train_with_markers.csv',
    '/content/drive/MyDrive/train_no_markers.csv',
    '/content/drive/MyDrive/valid_with_markers.csv',
    '/content/drive/MyDrive/valid_no_markers.csv',
    '/content/drive/MyDrive/test_with_markers.csv',
    '/content/drive/MyDrive/test_no_markers.csv'
]

# Read CSV files with chunksize and store in a list
data_files = [pd.read_csv(file, chunksize=10000) for file in file_paths]

# Create a list to hold data frames
data_frames = []

# Process each file's chunks and concatenate into a single DataFrame
for i, data_file in enumerate(data_files):
    chunks = []
    for chunk in data_file:
        chunks.append(chunk)

    # Concatenate chunks into a single DataFrame
    concatenated_df = pd.concat(chunks, ignore_index=True)

    # Define variable name dynamically and store in a list
    df_name = f"df_{i}"
    data_frames.append((df_name, concatenated_df))

df_train_with_markers = data_frames[0][1]
df_train_no_markers = data_frames[1][1]
df_valid_with_markers = data_frames[2][1]
df_valid_no_markers = data_frames[3][1]
df_test_with_markers = data_frames[4][1]
df_test_no_markers = data_frames[5][1]

# Access the concatenated DataFrames
print(f"df_train_with_markers: {df_train_with_markers.shape}")
print(f"df_train_no_markers: {df_train_no_markers.shape}")
print(f"df_valid_with_markers: {df_valid_with_markers.shape}")
print(f"df_valid_no_markers: {df_valid_no_markers.shape}")
print(f"df_test_with_markers: {df_test_with_markers.shape}")
print(f"df_test_no_markers: {df_test_no_markers.shape}")

df_train_with_markers: (60000, 7)
df_train_no_markers: (60000, 7)
df_valid_with_markers: (20000, 7)
df_valid_no_markers: (20000, 7)
df_test_with_markers: (20000, 7)
df_test_no_markers: (20000, 7)


In [ ]:
datasets = DatasetDict({
    "train": Dataset.from_pandas(df_train_no_markers),
    "validation": Dataset.from_pandas(df_valid_no_markers),
    "test": Dataset.from_pandas(df_test_no_markers)
})


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

checkpoint = "caseyhahn/gpt2-finetuned-genius-lyrics-updated-data"
model = TFAutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at caseyhahn/gpt2-finetuned-genius-lyrics-updated-data.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
def next_lyric_generate(prior_lyrics, tokenizer, model, token_length):
  input_ids = tokenizer.encode(prior_lyrics, return_tensors="tf")
  generated_text_outputs = model.generate(
    input_ids,
    min_length=len(input_ids[0,]) + token_length,
    max_length=len(input_ids[0,]) + token_length+2,
    num_return_sequences=1,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=False,
    pad_token_id=tokenizer.eos_token_id
    )
  generated_text_decoded = tokenizer.decode(generated_text_outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  words_input = prior_lyrics.split()
  words_output = generated_text_decoded.split()
  resulting_text = words_output[len(words_input):]
  resulting_text = ' '.join(resulting_text)
  return resulting_text

In [ ]:
def text_to_data(songs, model, tokenizer, num_songs=None, print_statement = False):
    lyric = []
    sections = []
    lyric_generated = []
    for i, song in enumerate(songs):
        if num_songs != None:
          if i >= num_songs:
            break
        section = 'missing'
        lyrics = song['lyrics'].split('\n')
        prior_lyrics = " "
        for i,line in enumerate(lyrics):
            if print_statement:
              print(f"actual line {i}:{line} ")
            if line == '':
                if print_statement:
                    print(f"predicted line {i}: ")
                continue
            elif (line[0] == '['):
                if print_statement:
                    print(f"predicted line {i}: ")
                if 'chorus' in line.lower():
                    section = 'Chorus'
                elif 'bridge' in line.lower():
                    section = 'Bridge'
                elif 'verse' in line.lower():
                    section = 'Verse'
            else:
              tokens_next_line = len(lyrics[i].split())
              next_line_generation = next_lyric_generate(prior_lyrics,tokenizer,model,tokens_next_line)
              if print_statement:
                print(f"predicted line {i}: {next_line_generation}")
              lyric_generated.append(next_line_generation)
              prior_lyrics += line + " "
              lyric.append(line)
              sections.append(section)
              if print_statement:
                print(" ")
    return lyric, lyric_generated, sections

In [ ]:
lyric, lyric_generated,sections = text_to_data(datasets['test'], model,tokenizer, num_songs=10)


In [ ]:
bleurt_score = bleurt.compute(predictions=lyric_generated, references=lyric)
individual_bleurt_scores = bleurt_score['scores']
print(f"bluert score: {np.mean(individual_bleurt_scores)}")


bluert score: -1.395632702503048


In [ ]:
individual_bleurt_scores

[-1.5096371173858643,
 -1.733971118927002,
 -1.5364538431167603,
 -1.4930622577667236,
 -1.054994821548462,
 -1.583678960800171,
 -1.5062716007232666,
 -1.1289993524551392,
 -1.1427497863769531,
 -1.323271632194519,
 -1.063493251800537,
 -1.7188234329223633,
 -1.5819228887557983,
 -1.7007269859313965,
 -1.4313468933105469,
 -1.3602423667907715,
 -1.4335589408874512,
 -1.4204516410827637,
 -1.0183703899383545,
 -0.7364049553871155,
 -1.169739007949829,
 -1.7224647998809814,
 -1.4276212453842163,
 -1.026539921760559,
 -1.512127161026001,
 -1.2861309051513672,
 -1.577447533607483,
 -1.512118935585022,
 -1.333612084388733,
 -1.0111452341079712,
 -1.076948642730713,
 -1.6038012504577637,
 -1.037250280380249,
 -1.7190961837768555,
 -1.7008129358291626,
 -1.1772634983062744,
 -0.7438672184944153,
 -1.6355196237564087,
 -1.1887269020080566,
 -0.966265857219696,
 -1.714142084121704,
 -1.4049404859542847,
 -1.1483426094055176,
 -1.1418551206588745,
 -1.4742854833602905,
 -1.3787736892700195,
 -1